<h1>Gaussian Mixture Model</h1>

# 0. EM

# 0.0 EM推导

参数估计首先想到极大似然估计。

假设模型的观测数据$Y$，隐变量（不可观测数据）$Z$，参数为$\theta$，完全数据$(Y, Z)$的联合概率分布$P(Y, Z|\theta)$。要估计参数$\theta$，首先计算观测数据关于参数的log似然。

> $l(\theta|Y) = \log p(Y|\theta)$

考虑完全数据的log似然，
> $\log p(Y, Z|\theta) = \log p(Y|\theta) p(Z|Y, \theta)
= \log p(Y|\theta) + \log p(Z|Y, \theta)$

从而$Y$的log似然，

> $\log p(Y|\theta) = \log p(Y,Z|\theta) - \log p(Z|Y, \theta)$

引入$Z$的另一个分布$q(Z)$，两边同时乘以$\sum_{Z} q(Z)$，

> $\sum_{Z} q(Z) \log P(Y|\theta) = \sum_{Z} q(Z) \left(\log p(Y,Z|\theta) - \log p(Z|Y, \theta)\right)$

> $\log p(Y|\theta) = \sum_{Z} q(Z) \left( \log p(Y,Z|\theta) - \log q(Z) + \log q(Z) - \log p(Z|Y, \theta) \right)$

> $ = \sum_Z q(Z) \left( \log \frac{p(Y,Z|\theta)} {q(Z)} -  
\frac{\log p(Z|Y, \theta)}{q(Z)} \right)$

> $ = \sum_Z q(Z) \log \frac{p(Y,Z|\theta)} {q(Z)} - \sum_Z q(Z) \frac{\log p(Z|Y, \theta)}{q(Z)} $

> $ = \mathcal{L}(q, \theta) + \mathbb{KL}[q||p]$

> $\mathcal{L}(q, \theta) = \sum_Z q(Z) \log \frac{p(Y,Z|\theta)} {q(Z)} 
= \mathbb{E}_q [\log p(Y,Z|\theta)]$

> $\mathbb{KL}[q||p] =  - \sum_Z q(Z) \frac{\log p(Z|Y, \theta)}{q(Z)}$

考虑不完全$Y$的数据的log似然，从完全数据$(Y,Z)$的log似然分解开始，然后引入隐变量的另一个分布$q(Z)$。

> $\log p(Y|\theta) = \mathcal{L}(q, \theta) + \mathbb{KL}[q||p]$

> $ >= \mathcal{L}(q, \theta)$

$\mathcal{L}(q, \theta)$是$\log p(Y|\theta)$的下界，当$\mathbb{KL}[q||p] = 0$时，$\mathcal{L}(q, \theta)$取得最大值，等于$\log p(Y|\theta)$，此时$q(Z) = p(Z|Y,\theta)$。

## 0.1 EM步骤

EM算法是迭代算法，分E-步与M-步。

+ E-步：根据$\theta^{old}$计算完全数据关于隐变量的期望$\mathbb{E}_{p(Z|Y, \theta^{old})} [\log p(Y,Z|\theta)]$

当$q(Z) = p(Z|Y, \theta^{old})$，

> $\mathcal{L}(q, \theta) = \sum_Z  p(Z|Y, \theta^{old}) \log \frac{p(Y,Z|\theta)} { p(Z|Y, \theta^{old})} 
 = \mathbb{E}_{p(Z|Y, \theta^{old})} [\log \frac{p(Y,Z|\theta)} { p(Z|Y, \theta^{old})}]$

需要计算$p(Z|Y, \theta^{old})$，根据贝叶斯规则，

> $p(Z|Y, \theta^{old}) = 
\frac{p(Y|Z, \theta^{old})p(Z|\theta)}
{\sum_Z p(Y|Z, \theta^{old})p(Z|\theta)}$

 
+ M-步：最大化$\mathbb{E}_{p(Z|Y, \theta^{old})} [\log \frac{p(Y,Z|\theta)} { p(Z|Y, \theta^{old})}] $，计算得到$\theta^{new}$

> $\theta^{new} = \arg\max_{\theta} \mathbb{E}_{p(Z|Y, \theta^{old})} [\log \frac{p(Y,Z|\theta)} { p(Z|Y, \theta^{old})}] $

> $ = \arg\max_{\theta} \mathbb{E}_{p(Z|Y, \theta^{old})} [\log p(Y,Z|\theta) ]$

此时，$q(Z) = p(Z|Y, \theta^{new})$。

所以需要在E-步计算$\mathbb{E}_{p(Z|Y, \theta^{old})} [\log p(Y,Z|\theta)]$。

重复E-步与M-步，直到满足停止迭代条件。

# 1. GMM

## 1.0 问题定义

假设数据由$K$个高斯分布$\mathcal{N}(\mu_k, \sigma_k^2)$组成，

> $p(y|\theta) = \sum_{k=1}^{K} a_k \mathcal{N} (\mu_k, \sigma_k^2)$

> $a_k >=0$

> $\sum_{k=1}^{K} a_k = 1$

观测数据是$y_i, i=1, 2, ..., N$；

隐含变量是每一个样本$y_i$属于$k$个高斯分布中的哪一个$\gamma_{ik}= p(z_k=1|y_i,\theta)$；

参数是$K$个高斯分布的均值与方差$\theta=\{a_k, (\mu_k, \sigma_k^2), k=1,2,...,K\}$。

## 1.1 EM求解

+ E-步

计算完全数据关于隐含变量的log似然的期望$\mathbb{E}_{\gamma} [\log p(Y,Z|\theta)]$；

首先计算$p(Z|Y,\theta^{old})$，

> $\gamma_{ik} = p(z_k=1|y_i,\theta)$

> $ = \frac{p(z_k=1)p(y_i|z_k=1,\theta)}
{\sum_{j=1}^{K}p(z_j=1)p(y_i|z_j=1,\theta)}$

> $ = \frac{ a_k \mathcal{N}(y_i|\mu_k, \sigma_k^2) }
{\sum_{j=1}^{K} a_j \mathcal{N}(y_i|\mu_j, \sigma_j^2) }
$

然后计算$\mathbb{E}_{\gamma} [\log p(Y,Z|\theta)]$，

完整数据的似然函数为，

> $L(\theta|Y) = 
\prod_{i=1}^N p(y_i, \gamma_{i1}, \gamma_{i2}, ..., \gamma_{iK}| \theta) $

> $ = \prod_{i=1}^N \prod_{k=1}^K
\left\{ a_k \mathcal{N}(y_i|\mu_k, \sigma_k^2) \right\} ^{\gamma_{ik}}
$

完整数据的log似然为，

> $l(\theta|Y) = \log L(\theta|Y) = \sum_{i=1}^N \sum_{k=1}^K
\gamma^{ik}
\log \left\{
a_k \mathcal{N}(y_i|\mu_k, \sigma_k^2)
\right\}$

从而，完整数据log似然关于隐变量的期望为，

> $\mathbb{E}_{\gamma}[l(\theta|Y)] = 
\mathbb{E}\left[
\sum_{i=1}^N \sum_{k=1}^K
\gamma_{ik} \left(
\log a_k + 
\log \mathcal{N}(\mu_k, \sigma_k^2)
\right)
\right]$

> $ = \mathbb{E} \left[
\sum_{i=1}^{N} \sum_{k=1}^{K} \gamma_{ik}\left( 
\log a_k + \log (\frac{1}{\sqrt{2\pi}\sigma_k}exp\{-\frac{(y_i-\mu_k)^2}{2\sigma_k^2}\} \right) \right]$

> $ = \mathbb{E} \left[
\sum_{i=1}^{N} \sum_{k=1}^{K} \gamma_{ik}\left( 
\log a_k - \log (\sqrt{2\pi}) - \log\sigma_k - \frac{(y_i-\mu_k)^2}{2\sigma_k^2} 
\right) \right]$

> $ = \sum_{i=1}^N \sum_{k=1}^K 
\mathbb{E}(\gamma_{ik})
\left( 
\log a_k - \log (\sqrt{2\pi}) - \log\sigma_k - \frac{(y_i-\mu_k)^2}{2\sigma_k^2} 
\right)
$

> $\hat{\gamma_{ik}} = \mathbb{E}(\gamma_{ik}) = \gamma_{ik}$

> $ = p(z_k=1|y_i,\theta)$

> $ = \frac{p(z_k=1)p(y_i|z_k=1,\theta)}
{\sum_{j=1}^{K}p(z_j=1)p(y_i|z_j=1,\theta)}$

> $ = \frac{ a_k \mathcal{N}(y_i|\mu_k, \sigma_k^2) }
{\sum_{j=1}^{K} a_j \mathcal{N}(y_i|\mu_j, \sigma_j^2) }
$

+ M-步
分别计算$\mathbb{E}_{\gamma}[l(\theta|Y)]$关于$\mu_k,\sigma_k$的偏导数，令其等于0，得到参数的极大值，

> $\frac{\partial{\mathbb{E}_{\gamma}[l(\theta|Y)]}}{\partial{\mu_k}} 
= \sum_{i=1}^{N} \gamma_{ik} \left( - \frac{\mu_k - y_i}{\sigma_k^2} \right) = 0$

> $\mu_k = \frac{\sum_{i=1}^{N}\gamma_{ik}y_i}{\sum_{i=1}^{N}\gamma_{ik}}$

> $\frac{\partial{\mathbb{E}_{\gamma}[l(\theta|Y)]}}{\partial{\sigma_k}}
= \sum_{i=1}^N \gamma_{ik}\left( 
-\frac{1}{\sigma_k}-\frac{(y_i-\mu_k)^2}{2}\frac{-2}{\sigma_k^3}
\right) = 0$

> $\sigma_k^2 = \frac{\sum_{i=1}^{N}\gamma_{ik}(y_i-\mu_k)^2}{\sum_{i=1}^{N}\gamma_{ik}}$

$a_k$满足限制$\sum_{k=1}^K a_k = 1$，加入拉格朗日乘子，

> $\mathbb{E}_{\gamma}[l(\theta|Y)] + \lambda(\sum_{k=1}^K a_k - 1)$

对上式关于$a_k$求偏导，

> $\sum_{i=1}^N \frac{\gamma_{ik}}{a_k} + \lambda = 0$

等式两边各乘以$a_k$，

> $\lambda a_k = - \sum_{i=1}^N \gamma_{ik}$

对$k=1, 2, ..., K$分别计算上式，将$K$个等式相加，

> $\sum_{k=1}^K \lambda a_k = - \sum_{k=1}^{K} \sum_{i=1}^N \gamma_{ik}
= - \sum_{i=1}^N \left( \sum_{k=1}^K \gamma_{ik} \right)$

> $\lambda = -N $


> $a_k = \frac{1}{N} \sum_{i=1}^N \gamma_{ik}$